Huh: Why he used Tensorflow and Pytorch. Why not just one of them?

In [41]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from transformers import AdamW, BertForSequenceClassification, get_linear_schedule_with_warmup
from tqdm import tqdm, trange  #for progress bars
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image #for image rendering

In [42]:
%%capture
%pip install datasets

In [43]:
from datasets import load_dataset
xdf = load_dataset('csv', data_files="/content/drive/MyDrive/Scripts/30 Days Of Bert/Day06/train.csv")
xdf

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16800
    })
})

In [44]:
xdf = xdf['train'].train_test_split(test_size=0.2)
xdf

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 13440
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3360
    })
})

In [45]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
!nvidia-smi

Mon Feb 10 15:19:33 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   63C    P0             30W /   70W |     574MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Aha: BertTokenizer já faz tudo - Adiciona [CLS], [SEP] e transformar as palavras em números.

In [46]:
from transformers import BertTokenizer
from transformers import AutoTokenizer

model_ckpt="google-bert/bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

Aha: return_tensors já transformar em tensores

Aha: Os tensores ficam em inputs_ids

In [47]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, return_tensors="pt")

x_encoded = xdf.map(tokenize, batched=True, batch_size=None)
x_encoded

Map:   0%|          | 0/13440 [00:00<?, ? examples/s]

Map:   0%|          | 0/3360 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 13440
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3360
    })
})

In [48]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import AutoModel

model_ckpt = "bert-base-uncased"
num_labels = 2

model = (AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=num_labels).to(device))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}


In [55]:

from transformers import Trainer, TrainingArguments
batch_size = 64

logging_steps = len(xdf["train"]) // batch_size
model_name = f"{model_ckpt}-finetuned-emotion"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=2,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=True,
                                  log_level="error")


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer

trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=x_encoded["train"],
                  eval_dataset=x_encoded["test"],
                  tokenizer=tokenizer)
trainer.train();

<ipython-input-56-a78bd3a7c74f>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model=model, args=training_args,


Epoch,Training Loss,Validation Loss


In [ ]:
from datasets import load_dataset
val = load_dataset('csv', data_files="/content/drive/MyDrive/Scripts/30 Days Of Bert/Day10/test (4).csv")
val = val.remove_columns(["id"])
val